In [3]:
import numpy as np
from gymnasium.spaces import Space, Box
from poke_env.player import Gen8EnvSinglePlayer

from poke_env.environment.abstract_battle import AbstractBattle
from poke_env.player import (
    Gen8EnvSinglePlayer,
    MaxBasePowerPlayer,
    ObsType,
    RandomPlayer,
    SimpleHeuristicsPlayer,
    background_cross_evaluate,
    background_evaluate_player,
)

class SimpleRLPlayer(Gen8EnvSinglePlayer):
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle: AbstractBattle) -> ObsType:
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                )

        # We count how many pokemons have fainted in each team
        fainted_mon_team = len([mon for mon in battle.team.values() if mon.fainted]) / 6
        fainted_mon_opponent = (
            len([mon for mon in battle.opponent_team.values() if mon.fainted]) / 6
        )

        # Final vector with 10 components
        final_vector = np.concatenate(
            [
                moves_base_power,
                moves_dmg_multiplier,
                [fainted_mon_team, fainted_mon_opponent],
            ]
        )
        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = [-1, -1, -1, -1, 0, 0, 0, 0, 0, 0]
        high = [3, 3, 3, 3, 4, 4, 4, 4, 1, 1]
        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32,
        )

...

Ellipsis

In [ ]:
from gymnasium.utils.env_checker import check_env
from poke_env.player import RandomPlayer

opponent = RandomPlayer(battle_format="gen8randombattle")
test_env = SimpleRLPlayer(
    battle_format="gen8randombattle", opponent=opponent, start_challenging=True
)
check_env(test_env)
test_env.close()

2024-11-25 11:21:33,792 - SimpleRLPlayer 2 - ERROR - Unhandled exception raised while handling message:
>battle-gen8randombattle-228
|player|p2|RandomPlayer 2|2|
|teamsize|p1|6
|teamsize|p2|6
|gen|8
|tier|[Gen 8] Random Battle
|rule|Species Clause: Limit one of each Pokémon
|rule|HP Percentage Mod: HP is shown in percentages
|rule|Sleep Clause Mod: Limit one foe put to sleep
|rule|Illusion Level Mod: Illusion disguises the Pokémon's true level
|
|t:|1732530093
|start
|switch|p1a: Inteleon|Inteleon, L80, F|243/243
|switch|p2a: Coalossal|Coalossal, L88, M|100/100
|turn|1
Traceback (most recent call last):
  File "c:\Users\maasm\.virtualenvs\ws_7-R81_JQuk\lib\site-packages\poke_env\ps_client\ps_client.py", line 135, in _handle_message
    await self._handle_battle_message(split_messages)  # type: ignore
  File "c:\Users\maasm\.virtualenvs\ws_7-R81_JQuk\lib\site-packages\poke_env\player\player.py", line 351, in _handle_battle_message
    await self._handle_battle_request(battle)
  File "c: